In [ ]:
!pip install keras-ocr -q

In [ ]:
from datasets import load_dataset

import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.


Looking for /root/.keras-ocr/crnn_kurapan.h5


In [ ]:
!pip install -q datasets

In [ ]:
import pandas as pd

df=pd.read_csv("/content/ReportWithImage.csv")
# Remove duplicates

df = df.drop_duplicates()
df.to_json('train.jsonl', orient='records', lines=True)
train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import requests
import cv2
import numpy as np

# Function to download the image from a URL
def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:

        image_array = np.asarray(bytearray(response.content), dtype=np.uint8)

          # Read image using OpenCV
        image1 = cv2.imdecode(image_array, cv2.IMREAD_COLOR)


        return image1

    else:
        print("Failed to download image:", response.status_code)
        return None

# URL of the image you want to perform OCR on
image_url = "https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/0064_0246679576_03_WRI-L2_F009.png?raw=true"


    # Initialize the keras-ocr pipeline
pipeline = keras_ocr.pipeline.Pipeline()




Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [ ]:
def perform_ocr(image):

    # Perform OCR on the image
    prediction_groups = pipeline.recognize([image])
    # Extract text results
    text_results = []
    for predictions in prediction_groups:
        for text_result in predictions:
            text_results.append(text_result[0])
    return text_results

In [ ]:
imageUrl=[]
image_name=[]
# Map the format and populate the data dictionary
for example in train_dataset:
    imageUrl.append(f"https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/{example['image_name']}?raw=true")
    image_name.append(example['image_name'])

type(image_name)





list

In [ ]:
# List to store image names and text results
results_data = []
count =0
# Loop through each image data
for image_url in imageUrl:

    # Download the image from the URL
    image_content = download_image(image_url)
    if image_content is not None:
        # Perform OCR on the image
        text_results = perform_ocr(image_content)
        print(count)
        # Append image name and text results to results_data list
        results_data.append({"image_name": image_name[count], "text_results": text_results})
        count =count+1
print(count)
# Create a DataFrame from results_data
df = pd.DataFrame(results_data)

# Save the DataFrame to a CSV file
df.to_csv("ocr_results.csv", index=False)

print("OCR results saved to ocr_results.csv")

1/1 [==============================] - 4s 4s/step
0
1/1 [==============================] - 1s 690ms/step
1
1/1 [==============================] - 0s 416ms/step
2


KeyboardInterrupt: 

In [ ]:
import multiprocessing
results_data = []
count =0
# Define a function to process each image
def process_image(image_url_list, image_name_list, results_data, start_index):
    for i, (image_url, image_name) in enumerate(zip(image_url_list, image_name_list)):
        image_content = download_image(image_url)
        if image_content is not None:
            text_results = perform_ocr(image_content)
            results_data.append({"image_name": image_name, "text_results": text_results})
    print(f"Process {start_index} to {start_index + len(image_url_list)} completed.")

# Split the image URL and name lists into two halves
half_length = len(imageUrl) // 2
first_half_urls = imageUrl[:half_length]
first_half_names = image_name[:half_length]
second_half_urls = imageUrl[half_length:]
second_half_names = image_name[half_length:]

# Initialize a multiprocessing pool
pool = multiprocessing.Pool()

# Submit the processing of each half to the pool
pool.apply_async(process_image, args=(first_half_urls, first_half_names, results_data, 0))
pool.apply_async(process_image, args=(second_half_urls, second_half_names, results_data, half_length))

# Close the pool and wait for all processes to complete
pool.close()
pool.join()

print("All images processed.")


In [ ]:
# Create a DataFrame from results_data
df = pd.DataFrame(results_data)

# Save the DataFrame to a CSV file
df.to_csv("ocr_results.csv", index=False)

print("OCR results saved to ocr_results.csv")